# Further Refinement of raw Adjacency Matrices

In [2]:
cd Autochef/src/autochef/RecipeAnalysis/

/home/jonas/tmp/autochef_python3.8/Autochef/src/autochef/RecipeAnalysis


In [3]:
import sys
sys.path.append("../")

In [4]:
import dill
import numpy as np
import settings
import AdjacencyMatrix
from tqdm.autonotebook import tqdm
from Recipe import Ingredient
from ActionGroups import groups

In [5]:
m_act = dill.load(open("m_act_raw.dill", "rb"))
m_mix = dill.load(open("m_mix_raw.dill", "rb"))
m_base_act = dill.load(open("m_base_act_raw.dill", "rb"))
m_base_mix = dill.load(open("m_base_mix_raw.dill", "rb"))

## Grouping Actions

In [21]:
groups = {
     'place':None,
     'heat':'heat',
     'cook':'heat',
     'bake':'heat',
     'grill':'heat',
     'melt':'heat',
     'blend':None,
     'beat':'prepare',
     'cool':'cool',
     'brown':'heat',
     'cut':'prepare',
     'chill':'cool',
     'drain':None,
     'boil':'heat',
     'simmer':'heat',
     'pour':None,
     'freeze':'cool',
     'saute':'heat',
     'rinse':'prepare',
     'warm':'heat',
     'wash':'prepare',
     'knead':'prepare',
     'peel':'prepare',
     'parboil':'heat',
     'break':'prepare',
     'broil':'heat',
     'scorch':'heat',
     'skim':None,
     'fry':'heat',
     'refrigerate':'cool',
     'burn':'heat',
     'thicken':None,
     'grate':'prepare',
     'brush':'prepare',
     'open':'prepare',
     'crack':'prepare',
     'poach':'heat',
     'slice':'prepare',
     'whisk':None,
     'dice':'prepare',
     'marinate':None,
     'whip':None,
     'sour':None,
     'soak':None,
     'steam':'heat',
     'chop':'prepare',
     'mince':None,
     'mash':'prepare',
     'squeeze':'prepare',
     'wipe':'prepare',
     'thaw':'prepare',
     'curdle':'heat',
     'sweeten':None,
     'baste':None,
     'carve':None,
     'grind':'prepare',
     'debone':'prepare',
     'steep':None,
     'clarify':None,
     'macerate':'prepare',
     'spread':None,
     'crumple':'prepare',
     'braise':'heat',
     'gut':None,
     'bury':None,
     'slowcook':'heat'
}

* now refactor the matrices to new versions that only contain those groups

In [22]:
# create new matrices:
m_grouped_act = AdjacencyMatrix.adj_matrix()
m_grouped_mix = AdjacencyMatrix.adj_matrix(True)
m_grouped_base_act = AdjacencyMatrix.adj_matrix()
#m_grouped_base_mix = AdjacencyMatrix.adj_matrix(True)

In [23]:
c = m_act.get_csr()

label_acts, labels_ings = m_act.get_labels()
acts, ings = c.nonzero()
for i_act,j_ing in tqdm(zip(acts,ings), total=len(acts)):
    ing = Ingredient.from_json(labels_ings[j_ing])
    act = label_acts[i_act]
    
    grouped_ing = Ingredient(ing._base_ingredient)
    for a in ing._action_set:
        grouped_ing.apply_action(groups[a])
    
    grouped_act = groups[act]
    
    m_grouped_act.add_entry(grouped_act, grouped_ing.to_json(),c[i_act, j_ing])
    

100%|██████████| 526406/526406 [00:45<00:00, 11492.10it/s]


In [24]:
c = m_mix.get_csr()

labels_ings = m_mix.get_labels()
ings_a, ings_b = c.nonzero()
for i_ing,j_ing in tqdm(zip(ings_a,ings_b), total=len(ings_a)):
    ing_a = Ingredient.from_json(labels_ings[i_ing])
    ing_b = Ingredient.from_json(labels_ings[j_ing])
    
    grouped_ing_a = Ingredient(ing_a._base_ingredient)
    for a in ing_a._action_set:
        grouped_ing_a.apply_action(groups[a])
    
    grouped_ing_b = Ingredient(ing_b._base_ingredient)
    for a in ing_b._action_set:
        grouped_ing_b.apply_action(groups[a])
        
    m_grouped_mix.add_entry(grouped_ing_a.to_json(), grouped_ing_b.to_json(),c[i_ing, j_ing])

100%|██████████| 2119768/2119768 [01:47<00:00, 19659.80it/s]


In [25]:
c = m_base_act.get_csr()

label_acts, labels_ings = m_base_act.get_labels()
acts, ings = c.nonzero()
for i_act,j_ing in tqdm(zip(acts,ings), total=len(acts)):
    base_ing = labels_ings[j_ing]
    act = label_acts[i_act]
    
    grouped_act = groups[act]
    
    m_grouped_base_act.add_entry(grouped_act, base_ing,c[i_act,j_ing])

100%|██████████| 78963/78963 [00:01<00:00, 45320.79it/s]


In [26]:
dill.dump(m_grouped_act, file=open("m_grouped_act_raw.dill", 'wb'))
dill.dump(m_grouped_mix, file=open("m_grouped_mix_raw.dill", 'wb'))
dill.dump(m_grouped_base_act, file=open("m_grouped_base_act_raw.dill", 'wb'))